## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [2]:
!pip3 install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 118kB/s 
     |████████████████████████████████| 3.8MB 35.3MB/s 
     |████████████████████████████████| 450kB 22.6MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.


In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/greatlakes/Residency6/InternalLab/prices.csv')

### Check all columns in the dataset

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [11]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'],axis=1)

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [14]:
data.shape

(851264, 5)

In [0]:
data_1000 = data.loc[0:1000]

In [17]:
data_1000['volume'] = data_1000['volume'] / 1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
data_1000.sample(4)

,open,close,low,high,volume
229,125.500000,124.370003,124.120003,126.000000,0.6767
563,34.590000,34.189999,34.000000,34.869999,5.3201
401,44.490002,44.790001,44.490002,45.250000,0.7742
634,33.919998,33.610001,33.380001,33.919998,1.5722


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

data_Y = data_1000["volume"]
data_X = data_1000.drop(["volume"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_X,data_Y, test_size = 0.2, random_state = 10)

#### Convert Training and Test Data to numpy float32 arrays


In [123]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 194 to 265
Data columns (total 4 columns):
open     800 non-null float64
close    800 non-null float64
low      800 non-null float64
high     800 non-null float64
dtypes: float64(4)
memory usage: 31.2 KB


In [0]:
X_train = X_train.to_numpy().astype('float32')
X_test = X_test.to_numpy().astype('float32')
y_train = y_train.to_numpy().astype('float32')
y_test = y_test.to_numpy().astype('float32')

In [125]:
X_train[0:1]

array([[130.52, 128.73, 128.17, 131.42]], dtype=float32)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
#from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
X_train_normalized = preprocessing.normalize(X_train)
X_test_normalized = preprocessing.normalize(X_test)

#transformer = Normalizer()
#train_x_1 = transformer.fit_transform(X_train)

In [52]:
X_train_normalized[0:1]

array([[0.5030965 , 0.49619684, 0.4940383 , 0.50656563]], dtype=float32)

In [53]:
X_train_normalized.shape

(800, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [130]:
for i in range(100):
    
    w, b = train(X_train_normalized, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train_normalized, w, b)).numpy())

Current Loss on iteration 0 278.37766
Current Loss on iteration 1 276.0609
Current Loss on iteration 2 273.92584
Current Loss on iteration 3 271.9583
Current Loss on iteration 4 270.14493
Current Loss on iteration 5 268.47354
Current Loss on iteration 6 266.93332
Current Loss on iteration 7 265.51385
Current Loss on iteration 8 264.20557
Current Loss on iteration 9 262.9999
Current Loss on iteration 10 261.8888
Current Loss on iteration 11 260.86484
Current Loss on iteration 12 259.9211
Current Loss on iteration 13 259.05154
Current Loss on iteration 14 258.24994
Current Loss on iteration 15 257.51105
Current Loss on iteration 16 256.8304
Current Loss on iteration 17 256.20297
Current Loss on iteration 18 255.6247
Current Loss on iteration 19 255.09183
Current Loss on iteration 20 254.60065
Current Loss on iteration 21 254.14813
Current Loss on iteration 22 253.73096
Current Loss on iteration 23 253.34648
Current Loss on iteration 24 252.99222
Current Loss on iteration 25 252.6656
Curr

### Get the shapes and values of W and b

In [131]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.3895233]
 [1.3948809]
 [1.3757309]
 [1.4066744]]
Bias:
 [2.783558]


In [132]:
X_test_normalized[0].shape

(4,)

In [133]:
import numpy as np
np.expand_dims(X_test_normalized[0], axis=0).shape

(1, 4)

### Model Prediction on 1st Examples in Test Dataset

In [134]:
prediction(np.expand_dims(X_test_normalized[0], axis=0), w, b).numpy()

array([[5.567032]], dtype=float32)

In [135]:
y_test[0]

10.3077

In [137]:
loss(y_test[0],prediction(np.expand_dims(X_test_normalized[0], axis=0), w, b)).numpy()

22.473936

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_data = pd.read_csv('/gdrive/My Drive/greatlakes/Residency6/InternalLab/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [100]:
iris_data.head(2)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

iris_data['Species'] = le.fit_transform(iris_data['Species'])

In [91]:
list(le.classes_)

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [0]:
iris_data_one_hot_encoded = pd.get_dummies(iris_data, columns=['Species'])

In [109]:
iris_data_one_hot_encoded.sample(4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_0,Species_1,Species_2
33,34,5.5,4.2,1.4,0.2,1,0,0
111,112,6.4,2.7,5.3,1.9,0,0,1
5,6,5.4,3.9,1.7,0.4,1,0,0
126,127,6.2,2.8,4.8,1.8,0,0,1


### Splitting the data into feature set and target set

In [0]:
target = iris_data_one_hot_encoded[["Species_0","Species_1","Species_2"]]
features = iris_data_one_hot_encoded.drop(["Id","Species_0","Species_1","Species_2"], axis=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
#model.add(tf.keras.layers.Dense(4))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3,input_shape=(4,), activation='softmax'))

In [0]:
#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(features,target, test_size = 0.2, random_state = 10)

### Model Training 

In [149]:
model.fit(X_train_iris,y_train_iris,          
          validation_data=(X_test_iris,y_test_iris),
          epochs=100,
          batch_size=10)

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 0s 211us/sample - loss: 0.5129 - accuracy: 0.8000 - val_loss: 0.5772 - val_accuracy: 0.6000
Epoch 2/100
120/120 [==============================] - 0s 219us/sample - loss: 0.4993 - accuracy: 0.7750 - val_loss: 0.5366 - val_accuracy: 0.8000
Epoch 3/100
120/120 [==============================] - 0s 183us/sample - loss: 0.5111 - accuracy: 0.8167 - val_loss: 0.5478 - val_accuracy: 0.7000
Epoch 4/100
120/120 [==============================] - 0s 188us/sample - loss: 0.4929 - accuracy: 0.7833 - val_loss: 0.5336 - val_accuracy: 0.7667
Epoch 5/100
120/120 [==============================] - 0s 196us/sample - loss: 0.4987 - accuracy: 0.8000 - val_loss: 0.5372 - val_accuracy: 0.7333
Epoch 6/100
120/120 [==============================] - 0s 194us/sample - loss: 0.4889 - accuracy: 0.8333 - val_loss: 0.5377 - val_accuracy: 0.7000
Epoch 7/100
120/120 [==============================] - 0s 190us/sample - 

### Model Prediction

In [150]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [156]:
X_test_iris.shape

(30, 4)

In [157]:
prediction = model.predict(X_test_iris)

#Print prediction
print(prediction)

[[2.2566225e-02 6.8323326e-01 2.9420057e-01]
 [2.6325923e-03 3.1936601e-01 6.7800146e-01]
 [9.5510685e-01 4.4568695e-02 3.2445331e-04]
 [3.0539937e-02 5.5706555e-01 4.1239455e-01]
 [9.3341446e-01 6.5898858e-02 6.8661763e-04]
 [5.7799432e-02 6.5318578e-01 2.8901473e-01]
 [9.0625184e-03 4.9180940e-01 4.9912807e-01]
 [7.8766443e-02 6.6258359e-01 2.5864992e-01]
 [9.3507057e-01 6.4246647e-02 6.8277476e-04]
 [2.7088685e-02 4.9396789e-01 4.7894341e-01]
 [2.2149922e-02 4.3659803e-01 5.4125208e-01]
 [2.3446213e-03 2.3663661e-01 7.6101875e-01]
 [2.9035807e-02 6.0057396e-01 3.7039027e-01]
 [9.0537530e-01 9.3501136e-02 1.1235600e-03]
 [9.5287615e-01 4.6608739e-02 5.1512092e-04]
 [2.3850501e-03 3.3260635e-01 6.6500866e-01]
 [3.8469899e-02 5.3280663e-01 4.2872345e-01]
 [9.1191661e-01 8.6464636e-02 1.6186992e-03]
 [9.3802977e-01 6.1236691e-02 7.3350483e-04]
 [9.3824422e-01 6.1274920e-02 4.8082968e-04]
 [1.3686300e-03 2.8399026e-01 7.1464115e-01]
 [8.3562953e-04 1.9783203e-01 8.0133229e-01]
 [8.213965

In [159]:
predicted_num = np.argmax(prediction[0])

#Print the number
print(predicted_num)

1


In [168]:
y_test_iris[0:2]

,Species_0,Species_1,Species_2
87,0,1,0
111,0,0,1


In [167]:
predicted_num = np.argmax(prediction[1])

#Print the number
print(predicted_num)

2


### Save the Model

In [0]:
model.save('Iris_v1.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
#Initialize Sequential model
model_2 = tf.keras.models.Sequential()

#Normalize the data
model_2.add(tf.keras.layers.Dense(2,input_shape=(4,)))
#model_2.add(tf.keras.layers.BatchNormalization())
model_2.add(tf.keras.layers.Dense(3, activation='softmax'))
#Compile the model
model_2.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [165]:
model_2.fit(X_train_iris,y_train_iris,          
          validation_data=(X_test_iris,y_test_iris),
          epochs=100,
          batch_size=10)

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 0s 1ms/sample - loss: 2.4487 - accuracy: 0.4250 - val_loss: 1.0618 - val_accuracy: 0.6667
Epoch 2/100
120/120 [==============================] - 0s 200us/sample - loss: 1.0792 - accuracy: 0.4417 - val_loss: 0.9637 - val_accuracy: 0.2333
Epoch 3/100
120/120 [==============================] - 0s 194us/sample - loss: 0.9728 - accuracy: 0.2500 - val_loss: 0.9609 - val_accuracy: 0.2333
Epoch 4/100
120/120 [==============================] - 0s 198us/sample - loss: 0.9513 - accuracy: 0.2167 - val_loss: 0.9967 - val_accuracy: 0.2333
Epoch 5/100
120/120 [==============================] - 0s 199us/sample - loss: 0.9298 - accuracy: 0.3333 - val_loss: 0.8968 - val_accuracy: 0.2667
Epoch 6/100
120/120 [==============================] - 0s 189us/sample - loss: 0.9028 - accuracy: 0.3333 - val_loss: 1.0235 - val_accuracy: 0.3667
Epoch 7/100
120/120 [==============================] - 0s 192us/sample - lo

In [169]:
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 9         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
